In [2]:
import pandas as pd
import numpy as np

In [4]:
data=pd.read_csv("/kaggle/input/mydatafiles123/IMDB Dataset.csv")

In [5]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [6]:
data.shape

(50000, 2)

In [7]:
type(data)

pandas.core.frame.DataFrame

In [8]:
data.tail()

,review,sentiment
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative
49999,No one expects the Star Trek movies to be high...,negative


In [9]:
data["sentiment"].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [10]:
data.replace({"positive":1,"negative":0},inplace=True)

/tmp/ipykernel_34/2739597079.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.replace({"positive":1,"negative":0},inplace=True)


In [11]:
data.tail()

,review,sentiment
49995,I thought this movie did a down right good job...,1
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",0
49997,I am a Catholic taught in parochial elementary...,0
49998,I'm going to have to disagree with the previou...,0
49999,No one expects the Star Trek movies to be high...,0


In [12]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

2024-08-13 17:34:48.648106: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-13 17:34:48.648227: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-13 17:34:48.794595: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [13]:
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [14]:
train_data.shape

(40000, 2)

In [15]:
test_data.shape

(10000, 2)

In [17]:
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(train_data['review'])

In [18]:
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]), maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data["review"]), maxlen=200)

In [19]:
X_train

array([[2946, 3749, 1828, ...,  205,  351, 3856],
       [   3, 4004,  208, ...,   89,  103,    9],
       [   0,    0,    0, ...,    2,  710,   62],
       ...,
       [   0,    0,    0, ..., 1641,    2,  603],
       [   0,    0,    0, ...,  103,  125, 7285],
       [   0,    0,    0, ...,   70,   73, 2062]], dtype=int32)

In [20]:
X_test

array([[   0,    0,    0, ...,  995,  719,  155],
       [ 133,    6,  117, ...,    7, 9050, 8494],
       [   0,    0,    0, ...,   50, 1088,   96],
       ...,
       [   0,    0,    0, ...,  125,  200, 3241],
       [   0,    0,    0, ..., 1066,    1, 2305],
       [   0,    0,    0, ...,    1,  332,   27]], dtype=int32)

In [21]:
Y_train=train_data["sentiment"]
Y_test=test_data["sentiment"]

LSTM model building

In [24]:
model = Sequential()
model.add(Embedding(input_dim=10000, output_dim=128))
model.add(LSTM(units=128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(units=1, activation = "sigmoid"))

In [34]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (64, 200, 128)         │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (64, 128)              │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (64, 1)                │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,235,141 (16.16 MB)

 Trainable params: 1,411,713 (5.39 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,823,428 (10.77 MB)

In [27]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [28]:
model.fit(X_train, Y_train, batch_size=64, epochs=5, validation_split=0.2)

Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 126s 240ms/step - accuracy: 0.7109 - loss: 0.5507 - val_accuracy: 0.8512 - val_loss: 0.3652
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 120s 241ms/step - accuracy: 0.8419 - loss: 0.3765 - val_accuracy: 0.8409 - val_loss: 0.3674
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 119s 238ms/step - accuracy: 0.8746 - loss: 0.3059 - val_accuracy: 0.8654 - val_loss: 0.3305
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 120s 239ms/step - accuracy: 0.9036 - loss: 0.2488 - val_accuracy: 0.8525 - val_loss: 0.3572
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 119s 237ms/step - accuracy: 0.9161 - loss: 0.2248 - val_accuracy: 0.8521 - val_loss: 0.3644


In [29]:
loss, accuracy = model.evaluate(X_test, Y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 25s 79ms/step - accuracy: 0.8577 - loss: 0.3545


# **Building Predictive System**

In [32]:
model.save("model.h5")

In [33]:
import joblib
joblib.dump(tokenizer, "tokenizer.pkl")

['tokenizer.pkl']

In [43]:
def predictive_system(review):
    sequences = tokenizer.texts_to_sequences([review])
    padded_sequences = pad_sequences(sequences,maxlen=200)
    prediction = model.predict(padded_sequences)
    sentiment = "positive" if prediction[0][0]>0.5 else "negative"
    return sentiment

In [50]:
predictive_system("this movie was horrible")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step


'negative'

In [51]:
predictive_system("A trilling advanture with stunning visual")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step


'positive'

In [52]:
predictive_system("A visual masterpiece")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step


'positive'

In [53]:
predictive_system("this movie was long and slow")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step


'negative'